FLASK WEB SITE = DEMONSTRATE ENDPOINTS TO GET AND POST DATA TO PYTHON BACKEND

This will setup a flask app that has several actions available through the web API. 

The following are the request option URLs. 
* http://localhost:5000 = The basic index string.
* http://localhost:5000/new_message = The new message string.
* http://localhost:5000/api/data = Will get a JSON formatted data strting.

Key python module installs include:
* pip install flask
* pip install flask_cors
* pip install flask_wtf
* pip install flask-talisman
* pip install jupyter_server_proxy==4.1.2

In [ ]:
# Setup modules
from flask import Flask, jsonify, request
from flask_cors import CORS, cross_origin
from flask_wtf.csrf import CSRFProtect, generate_csrf
from datetime import datetime, timezone
import os
import json

# Define the Flask Application
app = Flask(__name__)

# Flask server port
FLASK_PORT = 5001

# SETUP FOR SECURE CSRF API CALLS (uncomment to use/explore)
# Note: Security is a big topic and can take time to understand and work with
# Set the secret key for CSRF protection
# app.config['SECRET_KEY'] = os.urandom(24)  # This generates a random secret key
# CORS(app)
# csrf = CSRFProtect(app)
# csrf.init_app(app)

# SETUP TO MINIMIZE SECURITY
# This will enable CORS for the whole app and is needed for the Javascript requests
CORS(app)

# Base URL endpoint
@app.route('/')
# Optionally can use the below statement to enable CORS for only this route/endpoint
# @cross_origin()
def index():
    current_time_utc = datetime.now(timezone.utc).isoformat()
    return f"Hello, Flask from Jupyter Notebook! (timestamp: {current_time_utc})"

# General new message string endpoint
@app.route('/new_message')
def new_message():
    current_time_utc = datetime.now(timezone.utc).isoformat()
    return f"Here is a new message example! (timestamp: {current_time_utc})"

# JSON data GET endpoint
# Note: Separate GET and POST routes for '/api/data'
@app.route('/api/data', methods=['GET'])
def get_data():
    current_time_utc = datetime.now(timezone.utc).isoformat()
    return jsonify({"message": f"Here is a JSON data example! (timestamp: {current_time_utc}"})

# JSON data POST endpoint that then writes POST data to a text file
@app.route('/api/data', methods=['POST'])
# Can add below decorator to explicitly exempt a route from CSRF
# @csrf.exempt
def post_data():
    # Get JSON data from the request
    data = request.get_json()
    
    # Ensure the data is received as JSON
    if data is None:
        return jsonify({"error": "JSON data is empty"}), 400
    
    # Write the data to a new data file with current UTC timestamp in name
    current_time_utc = datetime.now(timezone.utc)
    file_time = current_time_utc.strftime("%Y%m%d%H%M%S")
    directory = os.path.join(os.getcwd(), 'api_data')
    os.makedirs(directory, exist_ok=True)
    file_path = os.path.join(directory, f'{file_time}_api_data_received.json')
    try:
        with open(file_path, 'w') as json_file:
            json.dump(data, json_file)
        return jsonify({"message": "Data saved successfully!"}), 200
    except Exception as e:
        return jsonify({"error": str(e)}), 500
    
@app.route('/get_csrf_token', methods=['GET'])
def get_csrf_token():
    # Set token to simulated one for minimal security
    csrf_token = "ImZhNmZkNDE3Y2RkNDcwN2E4NTI3MTU1NGZjYzA0YmY0ZTFhOGQ4ODAi.X0H_hQ.Tkcnue__YZSH6V9n8xwSkI7xY5M"
    
    # Uncomment below to generate a CSRF token if working with CSRF security
    # csrf_token = generate_csrf()
    return jsonify({'csrf_token': csrf_token})


In [ ]:
from threading import Thread

# Create the function to call and run the flask web app
def run_app():
    app.run(host='127.0.0.1', port=FLASK_PORT)
    # Note: Above only allows localhost.  To allow all swap comments between above and below. 
    # app.run(host='0.0.0.0', port=5000)
    
# Create a new execution thread in the Jupyter instance to serve the previously created Flask web site. 
thread = Thread(target=run_app)
thread.start()